# Imports
All imports should be defined here to reduce clutter

In [ ]:
import numpy as np
import pygame
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import matplotlib.pyplot as plot

from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from generation.HaltedEvolution import HaltedEvolution

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Function Declarations

In [ ]:
def initializeHaltedEvolution():
    agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
    ]

    phenotype = ConfigurationDefaults.BEHAVIOR_VECTOR

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=30,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=100,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=15,
        simulation_lifespan=300,
        display_novelty=False,
        save_archive=False,
        show_gui=True
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen
    )

    halted_evolution = HaltedEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    return halted_evolution

In [ ]:
class BehaviorIdentificationModel(torch.nn.Module):
    def __init__(self, n_classes=2):
        super(BehaviorIdentificationModel, self,).__init__()

        self.n_classes = n_classes

        activation_function = torch.nn.LeakyReLU

        self.conv1 = torch.nn.Conv2d(1, 1, 5, stride=2, padding=2)
        self.activation1 = activation_function()
        self.conv2 = torch.nn.Conv2d(1, 1, 3, stride=2, padding=1)
        self.activation2 = activation_function()
        # self.conv3 = torch.nn.Conv2d(1, 1, 3, stride=2)
        # self.activation3 = activation_function()

        self.pooling = torch.nn.MaxPool2d(2, stride=2)
        self.flatten = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(3844, 400)
        # self.normalization1 = torch.nn.BatchNorm1d(400)
        self.activation5 = activation_function()

        self.dropout = torch.nn.Dropout(0.05)

        FEATURE_LAYER_SIZE = 100
        self.linear2 = torch.nn.Linear(400, FEATURE_LAYER_SIZE)
        self.activation6 = activation_function()
        self.classification_layer = torch.nn.Linear(FEATURE_LAYER_SIZE, self.n_classes)

        self.curr_feature_layer = torch.zeros(FEATURE_LAYER_SIZE)


    def forward(self, x):
        x = self.conv1(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.activation2(x)
        # print(x.size())
        # x = self.conv3(x)
        # x = self.activation3(x)
        # print(x.size())

        x = self.pooling(x)
        x = self.flatten(x)
        # print(x.size())
        x = self.linear1(x)


        # if self.training:
        #     x = self.normalization1(x)
        x = self.activation5(x)
        x = self.linear2(x)
        self.curr_feature_layer = x
        if self.training:
            x = self.dropout(x)

        x = self.activation6(x)
        x = self.classification_layer(x)
        return x

    def increaseClassCount(self):
        self.n_classes += 1
        self.classification_layer = torch.nn.Linear(100, self.n_classes)

# Weight Initialization

In [ ]:
def weights_init_uniform_rule(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.uniform_(m.weight)

def weights_init_normal_rule(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.normal_(m.weight, mean=0, std=1)

def weights_init_xavier_rule(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)

def weights_init_xavier_uniform_rule(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight, gain=torch.nn.init.calculate_gain('relu'))

# Test Halted Evolution + TensorBoard
The following initializes a possible evolution, simulates the genome, and outputs the resulting frame to tensorboard

In [ ]:
def testTensorBoardAndEvolutionNexting():
    # Writer will output to ./runs/ directory by default
    writer = SummaryWriter()

    evolution = initializeHaltedEvolution()
    evolution.evolve_config.lifespan = 1200
    evolution.setup()

    for i in range(60):
        frame, behavior_vector = evolution.next()
        frame = frame.astype(np.uint8)
        reshaped = np.reshape(frame, (1, 500, 500))

        # Tensorboard output
        writer.add_image('images', reshaped.astype(np.uint8), i, dataformats="CWH")

    writer.close()
    pygame.quit()

# testTensorBoardAndEvolutionNexting()

# Test Single Pass Through of the Network

In [ ]:
def testInputAndBackpropOfAlteredNetwork():
    evolution = initializeHaltedEvolution()
    evolution.setup()

    model = BehaviorIdentificationModel(n_classes=6)
    frame, _ = evolution.next()
    frame = frame.astype(np.uint8)
    reshaped = np.reshape(frame, (1, 500, 500))

    # Initialize a test loss function
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    tensor = torch.tensor(reshaped, dtype=torch.float32)
    y_hat1 = model.forward(tensor)
    print(y_hat1)

    y_truth = torch.tensor([[1, 0, 0, 0, 0, 0]], dtype=torch.float32)
    loss = loss_fn(y_hat1, y_truth)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.increaseClassCount()
    y_hat2 = model.forward(tensor)
    y_truth = torch.tensor([[1, 0, 0, 0, 0, 0, 0]], dtype=torch.float32)
    loss = loss_fn(y_hat2, y_truth)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(y_hat1, y_hat2)
    pygame.quit()
# testInputAndBackpropOfAlteredNetwork()

Get Number of trainable parameters in the network

In [ ]:
def printModelInformation():
    model_info = BehaviorIdentificationModel(n_classes=2)
    print(sum(p.numel() for p in model_info.parameters() if p.requires_grad))
    print(f"Model structure: {model_info}\n\n")
printModelInformation()

In [ ]:
data_transform_training = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomApply(
        transforms=[
            transforms.RandomAffine(
                degrees=(0, 180),
                translate=(0.0, 0.3),
                scale=(0.5, 1.0)
            ),
            transforms.RandomPerspective(
                distortion_scale=0.6,
                p = 0.3
            )
        ],
        p=1.0
    ),
    transforms.ToTensor(),
    transforms.Normalize(0.0, 1.0)
])

data_transform_testing = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

train_folder = torchvision.datasets.ImageFolder(root="./data/train", transform=data_transform_training)
test_folder = torchvision.datasets.ImageFolder(root="./data/test", transform=data_transform_testing)

train_folder = torch.utils.data.ConcatDataset([train_folder] * 8)

In [210]:
OOD_CLASS = 2

def custom_softmax(x, dim=0):
    exp_x = torch.exp(x)
    exp_x = torch.nan_to_num(exp_x, nan=0.0, neginf=1e-5)
    # print("Exp X: ", exp_x)
    sum_x = torch.sum(exp_x, dim=dim, keepdim=True)

    # print("Sum X: ", sum_x)
    softmax = exp_x / sum_x
    softmax = torch.nan_to_num(softmax, nan=0.0, neginf=0.0, posinf=1.0)
    # print("Custom Softmax: ", softmax)
    if torch.isnan(softmax).any():
        raise Exception("NAN element in Custom Softmax")
    return softmax

def threshold_softmax(x, dim=0, a=0.2):
    # print("THRESHOLDING SET")
    alpha = torch.tensor(a, device=device)

    # limits = torch.full(x.shape, 1000, device=device)
    log_exp_x = torch.exp(x)
    log_exp_x = torch.nan_to_num(log_exp_x, nan=0.0, neginf=1e-5, posinf=1e5)
    # log_exp_x = torch.minimum(log_exp_x, limits)
    # print("Log Exp X: ", log_exp_x)
    sum_x = torch.sum(log_exp_x, dim=dim, keepdim=True) + torch.exp(alpha)
    # print("Sum X: ", sum_x)
    softmax = log_exp_x / sum_x
    softmax = torch.nan_to_num(softmax, nan=0.0, neginf=0.0, posinf=1.0)
    # print("Threshold Softmax: ", softmax)
    if torch.isnan(softmax).any():
        raise Exception("NAN element in Threshold Softmax")
    return softmax

def custom_log_softmax(x, a=None):
    log_softmax = torch.log(custom_softmax(x))
    if torch.isnan(log_softmax).any():
        raise Exception("NAN element in Custom Log Softmax")
    return log_softmax

def threshold_log_softmax(x, a=0.2):
    log_softmax = torch.log(threshold_softmax(x, a=a))
    if torch.isnan(log_softmax).any():
        print(log_softmax)
        raise Exception("NAN element in Threshold Log Softmax")
    return log_softmax

def custom_softmax_loss(x, y_truth, dim=0):
    log_prob = -1.0 * custom_log_softmax(x)
    # print("Log Prob: ", log_prob)
    loss = log_prob.gather(dim, y_truth)
    # print("Gathering: ", loss)
    loss = loss.mean()
    # print("Loss: ", loss)
    if torch.isnan(loss).any():
        raise Exception("NAN element in Custom Softmax Loss")
    return loss

def entropic_open_set_loss(x, y_truth=None, ret_mean=False, threshold_softmax=False):
    entropic_loss = torch.zeros(len(x), device=device)
    softmax = custom_log_softmax
    if threshold_softmax:
        # print("THRESHOLDING SET")
        softmax = threshold_log_softmax

    for i, y_t in enumerate(y_truth):
        if y_t == OOD_CLASS:
            entropic_loss[i] = (-1/len(x[i])) * torch.sum(softmax(x[i], a=threshold_softmax))
        else:
            logsoft = (-1) * softmax(x[i], a=threshold_softmax)
            entropic_loss[i] = logsoft[y_truth[i]]

    if torch.isnan(entropic_loss).any():
        raise Exception("NAN element in Entropic Open Set Loss")
    if ret_mean:
        return entropic_loss.mean()
    return entropic_loss

def objectosphere_loss(x, y_truth=None, feature_layer=None, ret_mean=True, lamb=1e-2, zeta=2.0, threshold_softmax=0.0):
    entropic_loss = entropic_open_set_loss(x, y_truth=y_truth, threshold_softmax=threshold_softmax)
    objectosphere = torch.zeros(len(x), device=device)
    LAMBDA = lamb

    for i, y_t in enumerate(y_truth):
        if y_t == OOD_CLASS:
            o_loss = torch.log(torch.norm(feature_layer[i]) ** 2)
            objectosphere[i] = torch.nan_to_num(o_loss, neginf=-3, nan=0)
        else:
            ZETA = zeta
            dist_to_zero = ZETA - torch.norm(feature_layer[i])
            o_loss = torch.log(torch.pow(torch.max(dist_to_zero, torch.tensor(0, device=device)), 2))
            objectosphere[i] = torch.nan_to_num(o_loss, neginf=-3, nan=0)

    total_loss = entropic_loss + (LAMBDA * objectosphere)
    if torch.isnan(total_loss).any():
        print(total_loss)
        raise Exception("NAN element in Objectosphere Loss")
    if ret_mean:
        return total_loss.mean()
    return total_loss

def binary_entropy_calculation(p1, p2):
    entropy = -(p1 * np.log2(p1)) - (p2 * np.log2(p2))
    return entropy

In [217]:
test = torch.tensor([[2.4283, 2.7178], [-0.8, -0.22], [0.8, 0.22]], device=device)
y_truth = torch.tensor([1, 1, 2], device=device)
features = torch.rand((3,3), device=device)
# softmax_a = torch.tensor([[0.3813, 0.6187]])
# softmax_b = torch.tensor([[0.1939, 0.8061]])

# print("Test: ", test)
# print("Y Truth: ", y_truth)
# print("Features: ", features)
#
print("Custom Softmax: ", custom_softmax(test, dim=1))
print("Entropic: ", entropic_open_set_loss(test, y_truth=y_truth))
print("Entropic: ", entropic_open_set_loss(test, y_truth=y_truth, threshold_softmax=0.5))
print("Objectosphere: ", objectosphere_loss(test, y_truth=y_truth, feature_layer=features, ret_mean=False, lamb=0.3))
# print("Binary Entropy Calculation A: ", binary_entropy_calculation(softmax_a[0][0], softmax_a[0][1]))
# print("Binary Entropy Calculation B: ", binary_entropy_calculation(softmax_b[0][0], softmax_b[0][1]))



Custom Softmax:  tensor([[0.4281, 0.5719],
        [0.3589, 0.6411],
        [0.6411, 0.3589]], device='cuda:0')
Entropic:  tensor([0.5588, 0.4446, 0.7346], device='cuda:0')
Entropic:  tensor([0.6192, 1.2849, 1.1232], device='cuda:0')
Objectosphere:  tensor([0.5259, 0.1848, 0.9118], device='cuda:0')


# Unit Test for Softmax

In [ ]:
TESTS = 100
THRESHOLD = 1e-2
for i in range(TESTS):
    x = torch.rand(2, device=device)
    softmax = custom_softmax(x, dim=0)
    softsum = torch.sum(softmax)
    if  softsum > (1 + THRESHOLD) or softsum < (1 - THRESHOLD):
        print("Input: ", x)
        print("Softmax: ", softmax)
        print("Softsum: ", softsum)
        assert False
print(f"{TESTS} tests passed.")

In [218]:
# CUDA_LAUNCH_BLOCKING=1
# with torch.no_grad():
#     torch.cuda.empty_cache()

eval_folder = torchvision.datasets.ImageFolder(root="./data/eval", transform=data_transform_testing)
eval_loader = torch.utils.data.DataLoader(
    eval_folder,
    batch_size=4,
    shuffle=False
)

train_loader = torch.utils.data.DataLoader(
    train_folder,
    batch_size=2,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_folder,
    batch_size=10,
    shuffle=True
)

SEED = 2
torch.manual_seed(SEED)

CLIP_GRADIENT = False
CLIP_VAL = 0.25
OOD_CLASS = 2
WEIGHT_INIT_FN = None
OBJECTO_LAMBDA = 1e-2
OBJECTO_ZETA = 1.6
THRESHOLD_ALPHA = 0.0
LEARNING_RATE = 15e-4 # 4e-4
INCLUDE_EVOLUTION = True
HITL = False

epochs = 100
model = BehaviorIdentificationModel(n_classes=2).to(device)

if WEIGHT_INIT_FN:
    model.apply(WEIGHT_INIT_FN) # Init Weights

if INCLUDE_EVOLUTION:
    evolution = initializeHaltedEvolution()
    evolution.setup()

loss_fn = custom_softmax_loss
decay_rate = 0.0

writer = SummaryWriter(log_dir="experiments/latest")
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

span_control = torch.zeros(10)
span_A = torch.zeros(10)
span_B = torch.zeros(10)

for epoch in range(epochs):
    print(f"Epoch {epoch}")

    # Training
    size = len(train_loader.dataset)
    loss_sum = 0
    loss_max = 0.0
    loss_min = 50.0
    model.train()
    for batch, (X, y) in enumerate(train_loader):

        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        pred = model(X)

        # loss = torch.nn.CrossEntropyLoss()(pred, y)
        loss = objectosphere_loss(
            pred,
            y_truth=y,
            feature_layer=model.curr_feature_layer,
            ret_mean=False,
            lamb=OBJECTO_LAMBDA,
            zeta=OBJECTO_ZETA,
            threshold_softmax=THRESHOLD_ALPHA
        )
        loss_max = max(loss_max, max(loss))
        loss_min = min(loss_min, min(loss))
        loss = loss.mean()
        loss.backward()
        loss_sum += loss.item()
        if CLIP_GRADIENT:
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_VAL)
        optimizer.step()

    epoch_loss = loss_sum / len(train_loader)
    writer.add_scalar('Loss/Train', epoch_loss, global_step=epoch)
    print(f"Training loss: {epoch_loss:>7f}")
    print(f"Max Loss: {loss_max}")
    print(f"Min Loss: {loss_min}")

    if INCLUDE_EVOLUTION:
        for i in range(max(0, epoch + 1)):
            frame, _ = evolution.next()
            frame = frame.astype(np.uint8)
            reshaped = np.reshape(frame, (1, 500, 500))

            # plot.imshow(frame, cmap='Greys')

            X = torch.tensor(reshaped, dtype=torch.float32, device=device)
            out = model(X)
            print("Predication: ", out.argmax())
            if HITL:
                i = input("Actual Class? Input 2 for OOD")

            optimizer.zero_grad()
            loss = objectosphere_loss(
                out,
                y_truth=torch.tensor([2]),
                feature_layer=model.curr_feature_layer,
                ret_mean=False,
                lamb=OBJECTO_LAMBDA,
                zeta=OBJECTO_ZETA,
                threshold_softmax=THRESHOLD_ALPHA
            )
            print("HITL Loss: ", loss.mean())
            loss.backward()
            optimizer.step()

    # Testing (Only test every few epochs)
    if epoch % 1 == 0:
        model.eval()
        size = len(test_loader.dataset)
        num_batches = len(test_loader)
        test_loss, accuracy = 0, 0

        with torch.no_grad():
            for X, y in test_loader:

                X = X.to(device)
                y = y.to(device)

                pred = model(X)
                loss = torch.nn.CrossEntropyLoss()(pred, y)
                test_loss += loss.item()
                softy = custom_softmax(pred, dim=1)

                for i, row in enumerate(softy):
                    if row.argmax() == y[i]:
                        accuracy += 1

            print("Softy: ", softy)

            SampleControl, _ = train_loader.dataset[3]
            SampleControl = SampleControl.to(device)
            control_pred = model(SampleControl.unsqueeze(0))
            control_pred = custom_softmax(control_pred.cpu(), dim=1)
            control = binary_entropy_calculation(control_pred[0][0], control_pred[0][1])
            writer.add_scalar("Control Entropy", control, epoch)
            print("Control: ", control)
            span_control = torch.concat((span_control, control.unsqueeze(0)))

            # ID SAMPLE
            SampleA, _ = eval_loader.dataset[3]
            SampleA = SampleA.to(device)
            predA = model(SampleA.unsqueeze(0))
            predA = custom_softmax(predA.cpu(), dim=1)
            entropyA = binary_entropy_calculation(predA[0][0], predA[0][1])
            writer.add_scalar("ID-Entropy (Sample A)", entropyA, epoch)
            print("Sample-A: ", entropyA)
            span_A = torch.concat((span_A, entropyA.unsqueeze(0)))

            # OOD Sample
            SampleB, _ = eval_loader.dataset[2]
            SampleB = SampleB.to(device)
            predB = model(SampleB.unsqueeze(0))
            predB = custom_softmax(predB.cpu(), dim=1)
            entropyB = binary_entropy_calculation(predB[0][0], predB[0][1])
            writer.add_scalar("OOD-Entropy (Sample B)", entropyB, epoch)
            print("Sample-B: ", entropyB)
            span_B = torch.concat((span_B, entropyB.unsqueeze(0)))

        test_loss /= num_batches
        accuracy /= size
        print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        writer.add_scalar('Loss/Test', test_loss, global_step=epoch)
        writer.add_scalar('Accuracy/Test', accuracy, epoch)

    scheduler.step()

if INCLUDE_EVOLUTION:
    pygame.quit()

Hello World!
Epoch 0
Training loss: 0.666521
Max Loss: 0.9406033754348755
Min Loss: 0.45234331488609314
Predication:  tensor(0, device='cuda:0')
HITL Loss:  tensor(21.6752, device='cuda:0', grad_fn=<MeanBackward0>)
Softy:  tensor([[0.5262, 0.4738],
        [0.5261, 0.4739],
        [0.5261, 0.4739],
        [0.5261, 0.4739]], device='cuda:0')
Control:  tensor(0.9980)
Sample-A:  tensor(0.9980)
Sample-B:  tensor(0.9981)
Test Error: 
 Accuracy: 48.2%, Avg loss: 0.696258 

Epoch 1
Training loss: 0.666591
Max Loss: 0.8729397058486938
Min Loss: 0.4914611876010895
Predication:  tensor(1, device='cuda:0')
HITL Loss:  tensor(inf, device='cuda:0', grad_fn=<MeanBackward0>)
Predication:  tensor(0, device='cuda:0')
HITL Loss:  tensor(inf, device='cuda:0', grad_fn=<MeanBackward0>)
Softy:  tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], device='cuda:0')
Control:  tensor(nan)
Sample-A:  tensor(nan)
Sample-B:  tensor(nan)
Test Error: 
 Accuracy: 48.2%, Avg loss:      nan 

Epoc

/tmp/ipykernel_1751866/3571595926.py:103: RuntimeWarning: divide by zero encountered in log2
  entropy = -(p1 * np.log2(p1)) - (p2 * np.log2(p2))


Training loss:     inf
Max Loss: inf
Min Loss: 50.0


KeyboardInterrupt: 

.# Checkpointing

In [ ]:
pygame.quit()

In [ ]:
# Save the Model
import time
file_name = f"cp_{round(time.time())}"
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, f"checkpoints/{file_name}.pt")

In [ ]:
# Load the model
FILE_PATH = "cp_E_B"
checkpoint = torch.load(f"checkpoints/{FILE_PATH}.pt")

eval_model = BehaviorIdentificationModel(n_classes=2)

if optimizer is None:
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

eval_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
eval_model.eval()

In [ ]:
print("Starting")
import matplotlib.pyplot as plot

model.eval()
eval_folder = torchvision.datasets.ImageFolder(root="./data/eval", transform=data_transform_testing)
eval_loader = torch.utils.data.DataLoader(
    eval_folder,
    batch_size=4,
    shuffle=False
)

with torch.no_grad():
    X, _ = eval_loader.dataset[2]
    # X, _ = eval_loader.dataset[3]
    # X, _ = train_loader.dataset[28]

    plot.imshow(torch.squeeze(X), cmap='Greys')
    plot.show()

    X = X.to(device)

    pred = model(X.unsqueeze(0))
    # target = torch.tensor([y])
    print("Prediction: ", pred, "Target: ", None)

    # optimizer.zero_grad()
    #
    # test_loss = loss_fn(pred, target)
    # print("Loss: ", test_loss.item())

    softy = custom_softmax(pred, dim=1)
    print("SoftMaxed: ", softy)

print("Ending")

In [ ]:
eval_model = eval_model.to(device)
eval_model.eval()
evolution = initializeHaltedEvolution()
evolution.setup()

for i in range(10):
    frame, _ = evolution.next()
    frame = frame.astype(np.uint8)
    reshaped = np.reshape(frame, (1, 500, 500))

    plot.imshow(frame, cmap='Greys')

    X = torch.tensor(reshaped, dtype=torch.float32, device=device)
    out = eval_model(X)
    print("Out: ", out)
    n_out = torch.nn.functional.normalize(out, dim=1)
    softy = torch.nn.functional.softmax(n_out, dim=1).cpu().detach().numpy()
    entro = binary_entropy_calculation(softy[0][0], softy[0][1])

    print("Softmax: ", softy)
    print("Entropy: ", entro)

pygame.quit()